In [ ]:
import requests
from bs4 import BeautifulSoup

BASE_URL = "https://foodsubs.com/groups"

def get_category_links():
    res = requests.get(BASE_URL)
    if res.status_code != 200:
        print(f"Failed to retrieve {BASE_URL}. Status code: {res.status_code}")
        return []
    
    soup = BeautifulSoup(res.text, "html.parser")
    
    # Find the div with class 'hero-groups' which contains ingredient groups
    hero_groups = soup.find("div", class_="hero-groups")
    
    if not hero_groups:
        print("Could not find 'hero-groups' div. Structure may have changed.")
        return []
    
    # Find all the <a> tags with the ingredient links inside this div
    links = hero_groups.find_all("a", href=True)
    
    category_links = []
    for link in links:
        href = link['href']
        print(href)  # Print all hrefs to inspect the structure
        
        # Check if the href represents a valid category page (adjust logic if necessary)
        # For example, you could look for specific patterns like '/groups/', '/categories/', etc.
        if "/groups/" in href:
            full_url = href if href.startswith("http") else f"{BASE_URL}{href}"
            category_links.append(full_url)

    return list(set(category_links))  # Deduplicate to avoid any duplicate links

if __name__ == "__main__":
    category_links = get_category_links()
    print(f"Found {len(category_links)} category pages.")
    for link in category_links[:5]:  # Print first 5 links
        print(link)


/groups/accompaniments
/groups/baked-goods
/groups/baking-supplies
/groups/dairy
/groups/equipment
/groups/fats-oils
/groups/fish
/groups/flavorings
/groups/fruit
/groups/grain-products
/groups/grains
/groups/legumes-nuts
/groups/liquids
/groups/meats
/groups/miscellaneous
/groups/vegetables
/groups/vegetarian
Found 17 category pages.
https://foodsubs.com/groups/baked-goods
https://foodsubs.com/groups/fats-oils
https://foodsubs.com/groups/grains
https://foodsubs.com/groups/vegetarian
https://foodsubs.com/groups/grain-products


In [ ]:
import time
import re
import requests
from bs4 import BeautifulSoup
import json

def extract_ingredients_from_page(url):
    res = requests.get(url)
    if res.status_code != 200:
        print(f"Failed to retrieve {url}. Status code: {res.status_code}")
        return []
    
    soup = BeautifulSoup(res.text, "html.parser")

    data = []

    paragraphs = soup.find_all(['p', 'ul'])
    for p in paragraphs:
        text = p.get_text().strip()
        if not text:
            continue

        # Try to identify substitutions
        match = re.search(r"(?P<name>^[^:.]+?)\s*(?:[:\-–—])?\s*(?:Substitute[s]?:?\s*(?P<subs>.*?))(?:\.\s|$)", text, re.IGNORECASE)
        if match:
            ingredient = match.group("name").strip()
            subs = match.group("subs").strip()
            data.append({
                "ingredient": ingredient,
                "substitutes": subs,
                "may_include": re.findall(r"\(.*?may include.*?\)", text, re.IGNORECASE)
            })

    return data

def scrape_foodsubs():
    category_links = get_category_links()
    all_ingredients = []

    for link in category_links:
        print(f"Scraping {link}")
        try:
            ingredients = extract_ingredients_from_page(link)
            all_ingredients.extend(ingredients)
        except Exception as e:
            print(f"Failed to scrape {link}: {e}")
        time.sleep(1)  # Be respectful
    
    return all_ingredients



/groups/accompaniments
/groups/baked-goods
/groups/baking-supplies
/groups/dairy
/groups/equipment
/groups/fats-oils
/groups/fish
/groups/flavorings
/groups/fruit
/groups/grain-products
/groups/grains
/groups/legumes-nuts
/groups/liquids
/groups/meats
/groups/miscellaneous
/groups/vegetables
/groups/vegetarian
Scraping https://foodsubs.com/groups/baked-goods
Scraping https://foodsubs.com/groups/fats-oils
Scraping https://foodsubs.com/groups/grains
Scraping https://foodsubs.com/groups/vegetarian
Scraping https://foodsubs.com/groups/grain-products
Scraping https://foodsubs.com/groups/fruit
Scraping https://foodsubs.com/groups/vegetables
Scraping https://foodsubs.com/groups/dairy
Scraping https://foodsubs.com/groups/legumes-nuts
Scraping https://foodsubs.com/groups/flavorings
Scraping https://foodsubs.com/groups/liquids
Scraping https://foodsubs.com/groups/baking-supplies
Scraping https://foodsubs.com/groups/meats
Scraping https://foodsubs.com/groups/miscellaneous
Scraping https://foodsub

In [22]:
# Execute the scraping
results = scrape_foodsubs()

# Saving the results to a file (e.g., JSON)
with open('ingredient_substitutions.json', 'w') as f:
    json.dump(results, f, indent=4)

print("Results saved to ingredient_substitutions.json")

/groups/accompaniments
/groups/baked-goods
/groups/baking-supplies
/groups/dairy
/groups/equipment
/groups/fats-oils
/groups/fish
/groups/flavorings
/groups/fruit
/groups/grain-products
/groups/grains
/groups/legumes-nuts
/groups/liquids
/groups/meats
/groups/miscellaneous
/groups/vegetables
/groups/vegetarian
Scraping https://foodsubs.com/groups/baked-goods
Scraping https://foodsubs.com/groups/fats-oils
Scraping https://foodsubs.com/groups/grains
Scraping https://foodsubs.com/groups/vegetarian
Scraping https://foodsubs.com/groups/grain-products
Scraping https://foodsubs.com/groups/fruit
Scraping https://foodsubs.com/groups/vegetables
Scraping https://foodsubs.com/groups/dairy
Scraping https://foodsubs.com/groups/legumes-nuts
Scraping https://foodsubs.com/groups/flavorings
Scraping https://foodsubs.com/groups/liquids
Scraping https://foodsubs.com/groups/baking-supplies
Scraping https://foodsubs.com/groups/meats
Scraping https://foodsubs.com/groups/miscellaneous
Scraping https://foodsub

In [3]:
pip install BeautifulSoup4

  Using cached soupsieve-2.6-py3-none-any.whl.metadata (4.6 kB)
Using cached soupsieve-2.6-py3-none-any.whl (36 kB)
Note: you may need to restart the kernel to use updated packages.


In [24]:
import time
import re
import json
import requests
from bs4 import BeautifulSoup

BASE_URL = "https://foodsubs.com/groups"

# Step 1: Get links to the full ingredient lists in each group
def get_ingredient_links():
    res = requests.get(BASE_URL)
    if res.status_code != 200:
        print(f"Failed to retrieve {BASE_URL}. Status code: {res.status_code}")
        return []

    soup = BeautifulSoup(res.text, "html.parser")
    groups_menu = soup.find("div", class_="groups-menu")

    if not groups_menu:
        print("Could not find 'groups-menu' div.")
        return []

    ingredient_links = []
    anchors = groups_menu.find_all("a", class_="list-group-item-see-all")
    for a in anchors:
        href = a.get("href")
        if href:
            full_url = href if href.startswith("http") else f"https://foodsubs.com{href}"
            ingredient_links.append(full_url)

    print(f"Found {len(ingredient_links)} ingredient list pages.")
    return list(set(ingredient_links))  # Deduplicate

# Step 2: Visit each ingredient list page and extract substitution info
def extract_ingredients_from_page(url):
    res = requests.get(url)
    if res.status_code != 200:
        print(f"Failed to retrieve {url}. Status code: {res.status_code}")
        return [], []

    soup = BeautifulSoup(res.text, "html.parser")
    data = []
    ingredient_urls = []

    # Pull substitution info from paragraphs and lists
    paragraphs = soup.find_all(['p', 'ul'])
    for p in paragraphs:
        text = p.get_text().strip()
        if not text:
            continue

        match = re.search(r"(?P<name>^[^:.]+?)\s*(?:[:\-–—])?\s*(?:Substitute[s]?:?\s*(?P<subs>.*?))(?:\.\s|$)", text, re.IGNORECASE)
        if match:
            ingredient = match.group("name").strip()
            subs = match.group("subs").strip()
            data.append({
                "ingredient": ingredient,
                "substitutes": subs,
                "source_url": url
            })

    # ✅ Grab "Learn more" ingredient detail links
    anchors = soup.select("a.card-learn-more[href*='/ingredients/']")
    for a in anchors:
        href = a.get("href")
        if href:
            full_url = href if href.startswith("http") else f"https://foodsubs.com{href}"
            ingredient_urls.append(full_url)

    return data, list(set(ingredient_urls))  # Return both substitution data and ingredient links


# Step 3: Scrape all pages
def scrape_foodsubs():
    ingredient_pages = get_ingredient_links()
    all_ingredients = []
    all_detail_links = []

    for link in ingredient_pages:
        print(f"Scraping {link}")
        try:
            ingredients, detail_links = extract_ingredients_from_page(link)
            all_ingredients.extend(ingredients)
            all_detail_links.extend(detail_links)
        except Exception as e:
            print(f"Failed to scrape {link}: {e}")
        time.sleep(1)  # Be respectful to the server

    all_detail_links = list(set(all_detail_links))  # Deduplicate if you plan to use them
    print(f"\n🔎 Found {len(all_detail_links)} ingredient detail pages.")
    return all_ingredients


# Step 4: Save results to JSON
if __name__ == "__main__":
    results = scrape_foodsubs()

    with open('ingredient_substitutions.json', 'w') as f:
        json.dump(results, f, indent=4)

    print(f"\n✅ Scraped {len(results)} ingredients. Results saved to ingredient_substitutions.json")


Found 61 ingredient list pages.
Scraping https://foodsubs.com/groups/vegetables/tubers-corms?ingredients
Scraping https://foodsubs.com/groups/meats/veal?ingredients
Scraping https://foodsubs.com/groups/grains?ingredients
Scraping https://foodsubs.com/groups/fruit?ingredients
Scraping https://foodsubs.com/groups/vegetables/fruit-vegetables/dried-chili-peppers?ingredients
Scraping https://foodsubs.com/groups/flavorings/wine?ingredients
Scraping https://foodsubs.com/groups/meats/beef?ingredients
Scraping https://foodsubs.com/groups/meats/variety-meats?ingredients
Scraping https://foodsubs.com/groups/meats?ingredients
Scraping https://foodsubs.com/groups/fish/shellfish?ingredients
Scraping https://foodsubs.com/groups/baking-supplies/leavens?ingredients
Scraping https://foodsubs.com/groups/fish/shellfish/crabs-and-shrimp?ingredients
Scraping https://foodsubs.com/groups/accompaniments/fruit-preserves?ingredients
Scraping https://foodsubs.com/groups/liquids/alcohol/wine?ingredients
Scraping h

In [13]:
import asyncio
import nest_asyncio
from playwright.async_api import async_playwright

# Apply nest_asyncio to allow the event loop to be nested
nest_asyncio.apply()

async def scrape_foodsubs():
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        await page.goto("https://foodsubs.com/groups")
        await page.wait_for_selector("a.card-title")  # Wait for all links to load

        # Extract all the <a> tags with class 'card-title'
        links = await page.query_selector_all("a.card-title")

        for link in links:
            name = await link.inner_text()  # Get the text of the link (ingredient name)
            href = await link.get_attribute("href")  # Get the href attribute (ingredient URL)
            print(f"Ingredient: {name}, URL: {href}")

        await browser.close()

# Run the function within the existing event loop
await scrape_foodsubs()



Error: BrowserType.launch: Executable doesn't exist at /Users/chengling/Library/Caches/ms-playwright/chromium_headless_shell-1161/chrome-mac/headless_shell
╔════════════════════════════════════════════════════════════╗
║ Looks like Playwright was just installed or updated.       ║
║ Please run the following command to download new browsers: ║
║                                                            ║
║     playwright install                                     ║
║                                                            ║
║ <3 Playwright Team                                         ║
╚════════════════════════════════════════════════════════════╝

In [10]:
pip install asyncio

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# Set up the WebDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# Open the URL
url = "https://foodsubs.com/groups"
driver.get(url)

# Wait for the page to load completely (adjust time if necessary)
# You can also wait for a specific element to load, like a main section
try:
    # Example of waiting for a specific element to load using XPath
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//a[@class="card-title"]'))
    )
except Exception as e:
    print("Error while waiting for the page to load:", e)

# Initialize an empty list to store the data
substitutes = []

# Loop through each group section
# Adjust the XPath to target the correct part of the page
groups_section = driver.find_elements(By.XPATH, '//a[@class="card-title"]')

for group in groups_section:
    try:
        # Extract the group name (e.g., "Dairy" or "Fruits")
        group_name = group.find_element(By.XPATH, './/h3').text.strip()

        # Find all the substitute list items within this group
        ingredient_substitutes = group.find_elements(By.XPATH, './/ul/li')

        for substitute in ingredient_substitutes:
            substitute_name = substitute.text.strip()
            substitutes.append({
                'Group': group_name,
                'Substitute': substitute_name
            })

    except Exception as e:
        print("Error while extracting group data:", e)

# Convert the list of dictionaries to a pandas DataFrame
df = pd.DataFrame(substitutes)

# Save the data to a CSV file
df.to_csv('ingredient_substitutes.csv', index=False)

# Display the first few rows of the DataFrame
print(df.head())

# Close the WebDriver after scraping
driver.quit()


Error while waiting for the page to load: Message: 
Stacktrace:
0   chromedriver                        0x000000010146c2c8 chromedriver + 6197960
1   chromedriver                        0x00000001014638ea chromedriver + 6162666
2   chromedriver                        0x0000000100ee8de0 chromedriver + 417248
3   chromedriver                        0x0000000100f3a797 chromedriver + 751511
4   chromedriver                        0x0000000100f3a9b1 chromedriver + 752049
5   chromedriver                        0x0000000100f8a9b4 chromedriver + 1079732
6   chromedriver                        0x0000000100f609ed chromedriver + 907757
7   chromedriver                        0x0000000100f87cdb chromedriver + 1068251
8   chromedriver                        0x0000000100f60793 chromedriver + 907155
9   chromedriver                        0x0000000100f2cb25 chromedriver + 695077
10  chromedriver                        0x0000000100f2d791 chromedriver + 698257
11  chromedriver                        0

In [18]:
pip install pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 11.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 7.3 MB/s eta 0:00:00a 0:00:01m
Note: you may need to restart the kernel to use updated packages.
